# Game of Life

## Table of Contents

* [What is it?](#What-is-it?)
  * [The rules](#The-rules)
  * [Simplified rules](#Simplified-rules)
* [Getting started](#Getting-started)
  * [Defining a grid](#Defining-a-grid)
  * [Helper functions](#Helper-functions)
  * [Testing our grid](#Testing-our-grid)
    * [Creating-a-basic-grid-with-all-cells-off](#Creating-a-basic-grid-with-all-cells-off)
    * [Testing-the-contains-function---can-we-easily-check-if-a-cell-exists-on-a-given-board](#Testing-the-contains-function---can-we-easily-check-if-a-cell-exists-on-a-given-board)
    * [Let's-try-to-turn-cells-on-and-off-and-Let's-test-checking-the-state-of-a-cell](#Let's-try-to-turn-cells-on-and-off-and-Let's-test-checking-the-state-of-a-cell)
    * [Initialize-a-grid-by-passing-in-an-array](#Initialize-a-grid-by-passing-in-an-array)
    * [Initialize-a-board-by-passing-specific-coordinates-to-turn-on](#Initialize-a-board-by-passing-specific-coordinates-to-turn-on)
    * [Check-that-we-can-accurately-get-a-list-of-neighbors-for-a-given-cell](#Check-that-we-can-accurately-get-a-list-of-neighbors-for-a-given-cell)
    * [Let's-test-the-walk-function](#Let's-test-the-walk-function)
* [Implementing the game](#Implementing-the-game)
  * [Pseudocode](#Pseudocode)
  * [More-helper-functions](#More-helper-functions)
  * [Simple examples](#Simple-examples)
  * [Better examples](#Better-examples)
    * [Blinker](#Blinker)
    * [Beacon](#Beacon)



## What is it?

This is an implementation of [Conway's Game of Life](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life). 

[top](#Table-of-Contents)

### The rules

These are the rules (*from Wikipedia*):

```
The universe of the Game of Life is an infinite, two-dimensional orthogonal grid of square cells, each of which is in one of two possible states, alive or dead, (or populated and unpopulated, respectively). Every cell interacts with its eight neighbours, which are the cells that are horizontally, vertically, or diagonally adjacent. At each step in time, the following transitions occur:

Any live cell with fewer than two live neighbours dies, as if by underpopulation.
Any live cell with two or three live neighbours lives on to the next generation.
Any live cell with more than three live neighbours dies, as if by overpopulation.
Any dead cell with exactly three live neighbours becomes a live cell, as if by reproduction.
These rules, which compare the behavior of the automaton to real life, can be condensed into the following:

Any live cell with two or three live neighbors survives.
Any dead cell with three live neighbors becomes a live cell.
All other live cells die in the next generation. Similarly, all other dead cells stay dead.
The initial pattern constitutes the seed of the system. The first generation is created by applying the above rules simultaneously to every cell in the seed; births and deaths occur simultaneously, and the discrete moment at which this happens is sometimes called a tick. Each generation is a pure function of the preceding one. The rules continue to be applied repeatedly to create further generations.
```
[top](#Table-of-Contents)

### Simplified rules

The above logic can be simplified as follows:

We are primarily concerned with cells that will change state: 
* a cell that is ALIVE can become DEAD  
* a cell that is DEAD can become ALIVE

What are the conditions that cause a cell to change state?
* A cell that is ALIVE becomes DEAD if it has less than 2 live neighbors or more than 3 live neighbors.
* A cell that is DEAD becomes ALIVE if it has exactly 3 LIVE neighbors

[top](#Table-of-Contents)

## Getting started 

### Defining a grid

We will maintain state in an n x n matrix.

For an n x n array, our indices will go from 1 .. n, with (1,1) being the cell in the top-left corner and (n,n) being the cell in the bottom-right corner

```
    (1,1)   (1,2)  ...  (1,n)
    
    (2,1)   (2,2)  ...  (2,n)
    
      |       |            |
      
    (n,1)   (n,2)  ...  (n, n)
```

A particular cell in the matrix can have one of two states: ON (aka ALIVE) or OFF (aka DEAD). These can be represented as any constants of your choice.

[top](#Table-of-Contents)


### Helper functions

We'll create some helper methods to manage our game board.

There are a lot of helper methods, but the most interesting are described below. 

**__init__**

> For convenience, there will be several ways to initialize a board. 
> * If you provide the SIZE, you will get an n x n array (where n=SIZE) with all the cells initialized to the DEAD state.
> * You can provide INIT, which is an n x n array, representing the exact initial state of the game
> * Either SIZE or INIT must be provided for initialization.
> * User cnn also optionally specify a list of COORDS representing the cells that should be turned on (ie, set the state of the cells specified in COORDS to ALIVE). 

**turn_on, turn_off, flip**

> These methods adjust the state of a given cell. TURN_ON will change a cell to ALIVE. TURN_OFF will change a cell to DEAD. FLIP will change the cell to the opposite of its current state (ALIVE-> DEAD, or DEAD-> ALIVE)

**is_alive, is_dead**

> Test the current state of a given cell and return a boolean value

**_get_cell_above, _get_cell_below, _get_cell_left, _get_cell_right, _get_cell_upper_left_diagnol, _get_cell_upper_right_diagnol, _get_cell_lower_left_diagnol, _get_cell_lower_right_diagnol**

> These methods return a neighboring cell of a given cell. There are 8 possible neighbors that a cell can have, so there are 8 helper methods. 

**get_neighbors**

> This will return a list of neighbors for a given cell. If the LIVEONLY flag is set, this method will include only neighbors that have a current state of ALIVE. (LIVEONLY is False by default, meaning you get the full list of neighbors if that flag is not specified).

**walk**

> This is a generator that provides a convenient way to iterate over all the cells in the board. An optional CELL paramter can be provided which represents the starting cell (default is (1,1), ie the first cell in the upper-left corner of the board). Beginning with a given cell, the code will move to the right as far as possible, then move to the first cell in the column below the current cell. So cells will be returned from right to left, top to bottom (the same order that we read English). This method continues until it reaches the last cell in the grid.

[top](#Table-of-Contents)

**The full code for the Grid class is shown below**

In [1]:
class Grid:
        
    def __init__(self, size=0, init=None, coords=None, consts=None):
        # hook so that we can easily change the display constants
        self.ON, self.OFF = 'X', '_'
        
        if consts:
            self.ON = consts[0]
            self.OFF = consts[1]

        
        # set the size of our array
        if init:
            self.size = len(init)
        elif size > 0:
            self.size = size
        else:
            raise Exception("An error occurred initializing the grid")
        
        # initialize our array to all dead cells
        self.rows = rows = []
        for i in range(self.size):
            row = []
            for j in range(self.size):
                row.append(self.OFF)
            rows.append(row)
        self.grid = rows 
        
        # if an initial array was provided, copy the values into the grid
        if init:
            n = len(init); m = len(init[0])
            for i in range(n):
                for j in range(m):
                    self.grid[i][j] = init[i][j]
                    
            
        # if coords were provided, set these cells to ON
        if coords:
            for cell in coords:
                self.turn_on(cell)
    
    
    def __contains__(self, cell):
        # Note - we will use indices starting at 1 (not 0)
        x, y = cell
        return x > 0 and x <= self.size and y > 0 and y <= self.size
    
    
    def __str__(self):
        res = []
        for row in self.grid:
            res.append(" ".join(row))
        return "\n".join(res)
 

    def __len__(self):
        return self.size
    
 
    def _grid(self):
        return self.rows
    
    
    def _get_real_coord(self, cell):
        """Adjust since the internal array uses zero-based indexing"""
        x, y = cell
        return (x-1, y-1)
    
    
    def _toggle(self, cell, val):
        """Set a given cell to a specified state - ON or OFF """
        if self.__contains__(cell):
            (x, y) = self._get_real_coord(cell)
            self.grid[x][y] = val


    def _check_value(self, cell, val):
        """Return the value of a given cell"""
        if self.__contains__(cell):
            (x, y) = self._get_real_coord(cell)
            return self.grid[x][y] == val

        
    def turn_on(self, cell):
        """Turn on a cell (set state to ALIVE)"""
        self._toggle(cell, self.ON)

            
    def turn_off(self, cell):
        """Turn off a cell (set state to DEAD)"""
        self._toggle(cell, self.OFF)
        
        
    def flip(self, cell):
        """Change the state to the opposite value for a given cell"""
        if self.is_alive(cell):
            self.turn_off(cell)
        else:
            self.turn_on(cell)

        
    def is_alive(self, cell):
        """Check if a given cell is ALIVE"""
        return self._check_value(cell, self.ON)
    
    
    def is_dead(self, cell):
        """Check if a given cell iS DEAD"""
        return self._check_value(cell, self.OFF)
    
    
    """
    These next methods help easily get a neighboring cell from a given cell's position
    """
    def _get_cell_above(self, cell):
        (x, y) = cell
        return (x-1, y)

    
    def _get_cell_below(self, cell):
        (x, y) = cell
        return (x+1, y)   
    
    
    def _get_cell_left(self, cell):
        (x, y) = cell
        return (x, y-1)   

    
    def _get_cell_right(self, cell):
        (x, y) = cell
        return (x, y+1)   
    
    
    def _get_cell_upper_left_diagnol(self, cell):
        (x, y) = cell
        return (x-1, y-1)   
    
    
    def _get_cell_upper_right_diagnol(self, cell):
        (x, y) = cell
        return (x-1, y+1)     
    
    
    def _get_cell_lower_left_diagnol(self, cell):
        (x, y) = cell
        return (x+1, y-1) 
    
    
    def _get_cell_lower_right_diagnol(self, cell):
        (x, y) = cell
        return (x+1, y+1) 
    
    
    def get_neighbors(self, cell, liveonly=False):
        """
        Return a list of tuples representing all possible neighboring cells
        if LIVEONLY is set to True, only include neighbors who are alive in this list
        """        
        neighbors = []
        
        # get coordinates of all possible neighbors
        above = self._get_cell_above(cell)
        below = self._get_cell_below(cell)
        right = self._get_cell_right(cell)
        left = self._get_cell_left(cell)
        upper_left = self._get_cell_upper_left_diagnol(cell)
        upper_right = self._get_cell_upper_right_diagnol(cell)
        lower_left = self._get_cell_lower_left_diagnol(cell)
        lower_right = self._get_cell_lower_right_diagnol(cell)
        
        # for each of the above possible neighbors, only add to the list if that cell is part of our existing board
        # also only add if the liveonly flag is not set or the neighboring cell is ALIVE
        if self.__contains__(above) and (not liveonly or self.is_alive(above) ) :
            neighbors.append(above)
        if self.__contains__(below) and (not liveonly or self.is_alive(below) ):
            neighbors.append(below)
        if self.__contains__(right) and (not liveonly or self.is_alive(right) ):
            neighbors.append(right)
        if self.__contains__(left) and (not liveonly or self.is_alive(left) ):
            neighbors.append(left)
        if self.__contains__(upper_left) and (not liveonly or self.is_alive(upper_left) ):
            neighbors.append(upper_left)
        if self.__contains__(upper_right) and (not liveonly or self.is_alive(upper_right) ):
            neighbors.append(upper_right)
        if self.__contains__(lower_left) and (not liveonly or self.is_alive(lower_left) ):
            neighbors.append(lower_left)
        if self.__contains__(lower_right) and (not liveonly or self.is_alive(lower_right) ):
            neighbors.append(lower_right)
    
        return neighbors

    
    def walk(self, cell=(1,1)):
        """Given a starting cell, iterate over the remaining cells in the grid, moving right, and then down, row by row"""
        current_row = cell[0]
        current_cell = cell
        while current_row <= self.size:
            while self.__contains__(current_cell):
                yield current_cell
                current_cell = self._get_cell_right(current_cell)
            current_row += 1
            current_cell = (current_row, 1)
        
    

[top](#Table-of-Contents)

### Testing our grid

#### Creating a basic grid with all cells off

In [2]:
print("Creating a 5x5 grid\n")
grid5x5 = Grid(size=5) 
print(grid5x5)

print("\n\nCreating a 9x9 grid\n")
grid9x9 = Grid(size=9)
print(grid9x9)


Creating a 5x5 grid

_ _ _ _ _
_ _ _ _ _
_ _ _ _ _
_ _ _ _ _
_ _ _ _ _


Creating a 9x9 grid

_ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _


[top](#Table-of-Contents)

#### Testing the contains function - can we easily check if a cell exists on a given board

In [3]:
# Remember we're using indices starting from 1 (not 0), so (0,0) is not part of our grid
print("Testing 'contains' function:")
print("(3,3) in grid5x5 --> True", (3,3) in grid5x5  )
print("(5,5) in grid5x5 --> True", (5,5) in grid5x5  )
print("(7,8) in grid5x5 --> False",(7,8) in grid5x5 )
print("(0,0) in grid5x5 --> False",(0,0) in grid5x5 )

print("\n")
print("(3,3) in grid9x9 --> True", (3,3) in grid9x9  )
print("(5,5) in grid9x9 --> True", (5,5) in grid9x9  )
print("(7,8) in grid9x9 --> True",(7,8) in grid9x9 )
print("(0,0) in grid9x9 --> False",(0,0) in grid9x9 )

Testing 'contains' function:
(3,3) in grid5x5 --> True True
(5,5) in grid5x5 --> True True
(7,8) in grid5x5 --> False False
(0,0) in grid5x5 --> False False


(3,3) in grid9x9 --> True True
(5,5) in grid9x9 --> True True
(7,8) in grid9x9 --> True True
(0,0) in grid9x9 --> False False


[top](#Table-of-Contents)

#### Let's try to turn cells on and off and Let's test checking the state of a cell

In [4]:
grid5x5 = Grid(size=5) 

print("Testing 'turn on' function -- TURN ON middle cell:")
grid5x5.turn_on((3,3))
print(grid5x5)

print("\n\nTesting 'turn on' function -- TURN ON cell in bottom left corner:")
grid5x5.turn_on((5,1))
print(grid5x5)

print("\n\nTesting 'turn on' function -- TURN ON cell directly above bottom right corner:")
grid5x5.turn_on((4,5))
print(grid5x5)

print("\n\nTesting 'turn off' function -- TURN OFF cell directly in the middle:")
grid5x5.turn_off((3,3))
print(grid5x5)

print("\n\nCell (4,5) is alive? (Expect True)", grid5x5.is_alive((4,5)) )
print("\nCell (4,5) is dead? (Expect False)", grid5x5.is_dead((4,5)) )
print("\nCell (3,3) is alive? (Expect False)", grid5x5.is_alive((3,3)) )
print("\nCell (3,3) is dead? (Expect True)", grid5x5.is_dead((3,3)) )


Testing 'turn on' function -- TURN ON middle cell:
_ _ _ _ _
_ _ _ _ _
_ _ X _ _
_ _ _ _ _
_ _ _ _ _


Testing 'turn on' function -- TURN ON cell in bottom left corner:
_ _ _ _ _
_ _ _ _ _
_ _ X _ _
_ _ _ _ _
X _ _ _ _


Testing 'turn on' function -- TURN ON cell directly above bottom right corner:
_ _ _ _ _
_ _ _ _ _
_ _ X _ _
_ _ _ _ X
X _ _ _ _


Testing 'turn off' function -- TURN OFF cell directly in the middle:
_ _ _ _ _
_ _ _ _ _
_ _ _ _ _
_ _ _ _ X
X _ _ _ _


Cell (4,5) is alive? (Expect True) True

Cell (4,5) is dead? (Expect False) False

Cell (3,3) is alive? (Expect False) False

Cell (3,3) is dead? (Expect True) True


[top](#Table-of-Contents)

#### Initialize a grid by passing in an array

In [5]:
print("""
# Create this simple board
#        O X O  
#        O X O
#        O X O
#
""")
simple_grid = Grid(init= [['O','X','O']]*3)
print(simple_grid)

print("\nTurn on the cell in the top left corner")
simple_grid.turn_on((1,1))
print(simple_grid)



# Create this simple board
#        O X O  
#        O X O
#        O X O
#

O X O
O X O
O X O

Turn on the cell in the top left corner
X X O
O X O
O X O


[top](#Table-of-Contents)

#### Initialize a board by passing specific coordinates to turn on

In [6]:
print("""
# Create this simple board
#        O O O  
#        X X X
#        O O O
#
""")
simple_grid2 = Grid(size=3, coords=[(2,1),(2,2),(2,3)], consts=['X','O'])
print(simple_grid2)

print("\nTurn on the cell in the bottom right corner")
simple_grid2.turn_on((3,3))
print(simple_grid2)


# Create this simple board
#        O O O  
#        X X X
#        O O O
#

O O O
X X X
O O O

Turn on the cell in the bottom right corner
O O O
X X X
O O X


[top](#Table-of-Contents)

#### Check that we can accurately get a list of neighbors for a given cell

In [7]:
grid5x5 = Grid(5)
print(grid5x5, "\n")
print("Neighbors of (1,1): ", grid5x5.get_neighbors((1,1)), "\n"  )
print("Neighbors of (3,3): ", grid5x5.get_neighbors((3,3)), "\n"  )
print("Neighbors of (4,2): ", grid5x5.get_neighbors((4,2)), "\n"  )
print("Neighbors of (5,5): ", grid5x5.get_neighbors((5,5)), "\n"  )

print("\nTurn on neighboring cells of (3,3)")
for cell in grid5x5.get_neighbors((3,3)):
    grid5x5.turn_on(cell)
print(grid5x5, "\n")


grid5x5 = Grid(5)
print("\nTurn on neighboring cells of (2,1)")
for cell in grid5x5.get_neighbors((2,1)):
    grid5x5.turn_on(cell)
print(grid5x5, "\n")


print("\nAll neighbors of (3,2): ", grid5x5.get_neighbors((3,2)), "\n"  )
print("Neighbors of (3,2) (LIVEONLY): ", grid5x5.get_neighbors((3,2), liveonly=True), "\n"  )

_ _ _ _ _
_ _ _ _ _
_ _ _ _ _
_ _ _ _ _
_ _ _ _ _ 

Neighbors of (1,1):  [(2, 1), (1, 2), (2, 2)] 

Neighbors of (3,3):  [(2, 3), (4, 3), (3, 4), (3, 2), (2, 2), (2, 4), (4, 2), (4, 4)] 

Neighbors of (4,2):  [(3, 2), (5, 2), (4, 3), (4, 1), (3, 1), (3, 3), (5, 1), (5, 3)] 

Neighbors of (5,5):  [(4, 5), (5, 4), (4, 4)] 


Turn on neighboring cells of (3,3)
_ _ _ _ _
_ X X X _
_ X _ X _
_ X X X _
_ _ _ _ _ 


Turn on neighboring cells of (2,1)
X X _ _ _
_ X _ _ _
X X _ _ _
_ _ _ _ _
_ _ _ _ _ 


All neighbors of (3,2):  [(2, 2), (4, 2), (3, 3), (3, 1), (2, 1), (2, 3), (4, 1), (4, 3)] 

Neighbors of (3,2) (LIVEONLY):  [(2, 2), (3, 1)] 



[top](#Table-of-Contents)

#### Let's test the walk function

Remember, by using walk, we should be able to iterate over all the cells in a particular grid (from an optional starting cell), moving right and then down

In [8]:
grid = Grid(size=3, coords=[(2,1),(2,2),(2,3)], consts=['X','O'])
print(grid)
print("\nIterating over all the cells in this grid and displaying their state")
for cell in grid.walk():
    print(cell, "ALIVE" if grid.is_alive(cell) else "DEAD" )
    
print("\nIterating over the cells again, this time starting from the middle cell (2,2)")
for cell in grid.walk((2,2)):
    print(cell, "ALIVE" if grid.is_alive(cell) else "DEAD" )


O O O
X X X
O O O

Iterating over all the cells in this grid and displaying their state
(1, 1) DEAD
(1, 2) DEAD
(1, 3) DEAD
(2, 1) ALIVE
(2, 2) ALIVE
(2, 3) ALIVE
(3, 1) DEAD
(3, 2) DEAD
(3, 3) DEAD

Iterating over the cells again, this time starting from the middle cell (2,2)
(2, 2) ALIVE
(2, 3) ALIVE
(3, 1) DEAD
(3, 2) DEAD
(3, 3) DEAD


[top](#Table-of-Contents)

## Implementing the game

At this point, we have *almost* all the tools we need to implement Conway's Game of Life.

We just need to implement the actual logic of the game.

### Pseudocode

**The pseudocode is as follows:**
* Initialize a game board (aka grid)
* Loop forever (or for a specified number of iterations)
  * get the new state of the grid
  * display the grid

**How do we get the new state of the grid**
* Iterate over every cell in the grid
  * Check the number of live neighbors the current cell has
  * Update the state of that cell according to the rules
  
*Note - we cannot actually change the state of a given cell until we have examined every cell. If we update the state of a cell as we iterate, then the new state will affect our processing of the next cell, which would not be accurate. Thus, we will maintain the new cell states in a new grid, and replace the original grid with the new grid once we have processed all the cells.*

[top](#Table-of-Contents)

### More helper functions

Let's define a few new functions to encapsulate the above logic.


In [9]:
def update_cell_state(newgrid, oldgrid, cell):
    """
    A cell that is ALIVE becomes DEAD if it has less than 2 live neighbors or more than 3 live neighbors.
    A cell that is DEAD becomes ALIVE if it has exactly 3 LIVE neighbors
    """
    
    # get number of live neighbors for this cell
    num_live_neighbors = len(oldgrid.get_neighbors(cell, liveonly=True))
    
    # if the cell meets one of the criteria above, flip its value and store the new state on the new grid
    if (oldgrid.is_alive(cell) and (num_live_neighbors < 2 or num_live_neighbors > 3)) or \
       (oldgrid.is_dead(cell) and num_live_neighbors==3):
        newgrid.flip(cell)
        

def update_grid_state(grid):
    """Return a new grid representing the new state"""
    
    # initialize a new grid with the same values as the old grid
    newgrid = Grid(init=grid._grid(), consts=(grid.ON, grid.OFF))
    
    # iterate over every cell in our input grid, and store the new state of the cell in the new grid
    for cell in grid.walk():
        update_cell_state(newgrid, grid, cell)
        
    # return the newgrid    
    return newgrid


def play_game(grid,iters=5):
    print("Initial grid")    
    print(grid)

    for i in range(iters):
        grid = update_grid_state(grid)
        print(f"\nGrid after iteration {i+1}")
        print(grid)

[top](#Table-of-Contents)

### Simple examples

**EXAMPLE 1**

Here's a grid that will change state only once, and then it gets stuck in a stable state.

In [10]:
grid = Grid(size=3, coords=[(1,1),(2,2),(3,3),(1,3),(3,1)])
play_game(grid, iters=3)
    

Initial grid
X _ X
_ X _
X _ X

Grid after iteration 1
_ X _
X _ X
_ X _

Grid after iteration 2
_ X _
X _ X
_ X _

Grid after iteration 3
_ X _
X _ X
_ X _


[top](#Table-of-Contents)

**EXAMPLE 2**

Here's another stable example. This grid won't change state no matter how many iterations are run.

In [11]:
grid = Grid(size=4, coords=[(2,2),(3,2),(2,3),(3,3)])
play_game(grid, iters=3)

Initial grid
_ _ _ _
_ X X _
_ X X _
_ _ _ _

Grid after iteration 1
_ _ _ _
_ X X _
_ X X _
_ _ _ _

Grid after iteration 2
_ _ _ _
_ X X _
_ X X _
_ _ _ _

Grid after iteration 3
_ _ _ _
_ X X _
_ X X _
_ _ _ _


[top](#Table-of-Contents)

**EXAMPLE 3**

This is called an oscillator. This grid bounces between two states.

In [12]:
grid = Grid(size=5, coords=[(3,2),(3,3),(3,4)])
play_game(grid)

Initial grid
_ _ _ _ _
_ _ _ _ _
_ X X X _
_ _ _ _ _
_ _ _ _ _

Grid after iteration 1
_ _ _ _ _
_ _ X _ _
_ _ X _ _
_ _ X _ _
_ _ _ _ _

Grid after iteration 2
_ _ _ _ _
_ _ _ _ _
_ X X X _
_ _ _ _ _
_ _ _ _ _

Grid after iteration 3
_ _ _ _ _
_ _ X _ _
_ _ X _ _
_ _ X _ _
_ _ _ _ _

Grid after iteration 4
_ _ _ _ _
_ _ _ _ _
_ X X X _
_ _ _ _ _
_ _ _ _ _

Grid after iteration 5
_ _ _ _ _
_ _ X _ _
_ _ X _ _
_ _ X _ _
_ _ _ _ _


[top](#Table-of-Contents)

### Better examples

We will test our game using some of the patterns described on the [Wikipedia page](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life#Examples_of_patterns). 

For the examples below, I will use the clear_output function so that each iteration of the grid gets printed in the same place as the previous iteration. This will enable us to more easily observe how the state changes.

To make this work we need to modify our play_game function a bit.

In [13]:
from time import sleep
from IPython.display import clear_output, display

def play_game_inplace(grid,iters=5):
    print("\nInitial grid")    
    print(grid)
    sleep(1)

    for i in range(iters):
        grid = update_grid_state(grid)
        clear_output(wait=True)
        print(f"\nGrid after iteration {i+1}")
        print(grid)
        sleep(1)

#### Oscillators

##### Blinker

In [14]:
grid = Grid(size=5, coords=[(3,2),(3,3),(3,4)])
play_game_inplace(grid)


Grid after iteration 5
_ _ _ _ _
_ _ X _ _
_ _ X _ _
_ _ X _ _
_ _ _ _ _


[top](#Table-of-Contents)

##### Beacon

In [15]:
# left, bottom, top, right
box1 = [(5,3),(6,3),(7,3),(8,5),(8,6),(8,7),(3,5),(3,6),(3,7),(5,8),(6,8),(7,8)]
box2 = [(5,10),(6,10),(7,10),(8,11),(8,12),(8,13),(3,11),(3,12),(3,13),(5,15),(6,15),(7,15)]
box3 = [(11,3),(12,3),(13,3),(15,5),(15,6),(15,7),(10,5),(10,6),(10,7),(11,8),(12,8),(13,8)]
box4 = [(11,10),(12,10),(13,10),(15,11),(15,12),(15,13),(10,11),(10,12),(10,13),(11,15),(12,15),(13,15)]

coords = []
coords.extend(box1)
coords.extend(box2)
coords.extend(box3)
coords.extend(box4)

grid = Grid(size=17, coords=coords, consts=['O','_'])
play_game_inplace(grid, iters=22)


Grid after iteration 22
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
_ _ _ _ _ O _ _ _ _ _ O _ _ _ _ _
_ _ _ _ _ O _ _ _ _ _ O _ _ _ _ _
_ _ _ _ _ O O _ _ _ O O _ _ _ _ _
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
_ O O O _ _ O O _ O O _ _ O O O _
_ _ _ O _ O _ O _ O _ O _ O _ _ _
_ _ _ _ _ O O _ _ _ O O _ _ _ _ _
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
_ _ _ _ _ O O _ _ _ O O _ _ _ _ _
_ _ _ O _ O _ O _ O _ O _ O _ _ _
_ O O O _ _ O O _ O O _ _ O O O _
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
_ _ _ _ _ O O _ _ _ O O _ _ _ _ _
_ _ _ _ _ O _ _ _ _ _ O _ _ _ _ _
_ _ _ _ _ O _ _ _ _ _ O _ _ _ _ _
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _


[top](#Table-of-Contents)

## Final Thoughts

That's my implementation of the Game of Life. Thanks for checking this out. I hope you found this helpful. 